In [1]:
import pandas as pd
import requests, glob, math, io

location = ''
files = glob.glob(location + '*.xlsx')
output_file = location + 'v4-trade.csv'

imports_file = [file for file in files if 'import' in file.lower()][0]
exports_file = [file for file in files if 'export' in file.lower()][0]

print('Reading in files')

dfI = pd.read_excel(imports_file, 'Country by Commodity Import')   # create imports dataframe
dfE = pd.read_excel(exports_file, 'Country by Commodity Export')   # create exports dataframe

assert len(dfI.columns) == len(dfE.columns), 'number of columns does not match for the import files'

''' Funtions '''
def v4Integers(value):
    '''
    treats all values in v4 column as strings
    returns integers instead of floats for numbers ending in '.0'
    '''
    newValue = str(value)
    if newValue.endswith('.0'):
        newValue = newValue[:-2]
    return newValue

def TimeCorrector(value):
    '''
    Converts YYYYMMM into mmm-yy
    '''
    # quick check to make sure time is in expected format
    assert len(value) == 7, '{} is not an expected time format'.format(value)
    
    year = value[:4]
    month = value[4:].title()
    
    # check to make sure format is correct
    try:
        int(year)
    except:
        raise ValueError('First for characters of {} should be a year'.format(value))
        
    return month + '-' + year[2:]

def AllCodesFromCodelist(url):
    '''
    gets all the codes from a code list
    works around the size limit within the API
    '''
    codelist_dict = requests.get(url).json()
    # total number of codes
    total_count = codelist_dict['total_count'] 
    
    codes_label_dict = {}
    
    # if < 1000 codes, no iteration needed
    if total_count <= 1000:
        new_url = url + '?limit=1000'
        whole_codelist_dict = requests.get(new_url).json()
        for item in whole_codelist_dict['items']:
            codes_label_dict.update({item['code']:item['label']})
        
    # otherwise iterations are needed, API only has limit size of 1000
    else:
        number_of_iterations = math.ceil(total_count / 1000)
        offset = 0
        for i in range(number_of_iterations):
            new_url = url + '?limit=1000&offset={}'.format(offset)
            whole_codelist_dict = requests.get(new_url).json()
            for item in whole_codelist_dict['items']:
                codes_label_dict.update({item['code']:item['label']})
            offset += 1000
            
    return codes_label_dict  

sitc_url = 'https://api.beta.ons.gov.uk/v1/code-lists/sitc/editions/one-off/codes'
commodity_dict = AllCodesFromCodelist(sitc_url)
    
def CommodityLabels(value):
    # returns sitc labels from api
    return commodity_dict[value]

country_url = 'https://api.beta.ons.gov.uk/v1/code-lists/countries-and-territories/editions/one-off/codes'
country_dict = AllCodesFromCodelist(country_url)
    
def CountryLabels(value):
    # returns countries-and-territories labels from api
    return country_dict[value]

def DirectionLabel(value):
    lookup = {'IM':'Imports', 'EX':'Exports'}
    return lookup[value]

def NANRemover(value):
    # changes a 'nan' to ''
    if pd.isnull(value):
        return ''
    else:
        return value
    
def Get_Latest_Version():
    '''
    Pulls the latest trade v4 from CMD 
    '''
    editions_url = 'https://api.beta.ons.gov.uk/v1/datasets/trade/editions/time-series/versions'
    items = requests.get(editions_url + '?limit=1000').json()['items']

    # get latest version number
    latest_version_number = items[0]['version']
    assert latest_version_number == len(items), 'Get_Latest_Version for /trade/editions/time-series - number of versions does not match latest version number'
    # get latest version URL
    url = editions_url + "/" + str(latest_version_number)
    # get latest version data
    latest_version = requests.get(url).json()
    # decode data frame
    file_location = requests.get(latest_version['downloads']['csv']['href'])
    file_object = io.StringIO(file_location.content.decode('utf-8'))
    df = pd.read_csv(file_object, dtype=str)
    return df
    
''' Post Processing '''
print('DataBaking..')
df_list = []

num = 0
for source in (dfI, dfE):
    num += 1 # counter for feedback
    
    for date_col in source.columns[3:]:
        
        df_loop = pd.DataFrame()
        
        df_loop['v4_0'] = source[date_col]
        
        df_loop['mmm-yy'] = TimeCorrector(date_col)
        df_loop['Time'] = df_loop['mmm-yy']
        
        df_loop['uk-only'] = 'K02000001'
        df_loop['Geography'] = 'United Kingdom'
        
        df_loop['sitc'] = source['COMMODITY'].apply(lambda x: x.split(' ')[0].replace('/', '-'))
        df_loop['StandardIndustrialTradeClassification'] = df_loop['sitc'].apply(CommodityLabels)
        
        df_loop['countries-and-territories'] = source['COUNTRY'].apply(lambda x: x.split(' ')[0])
        df_loop['CountriesAndTerritories'] = df_loop['countries-and-territories'].apply(CountryLabels)
        
        df_loop['trade-direction'] = source['DIRECTION'].apply(lambda x: x.split(' ')[0])
        df_loop['Direction'] = df_loop['trade-direction'].apply(DirectionLabel)
        
        df_list.append(df_loop)
        # print so that you can see that the transform is moving
        print('Generated sub-dataframe for {} from source {} of 2.'.format(date_col, num))
        
df = pd.concat(df_list)

df['v4_0'] = df['v4_0'].apply(NANRemover) #changes any 'nan' to ''
df['v4_0'] = df['v4_0'].apply(v4Integers) #changes floats to string-integers

previous_df = Get_Latest_Version()
new_df = pd.concat([previous_df, df]).drop_duplicates()
            
new_df.to_csv(output_file, index=False)
print('Transform complete!')

Reading in files
DataBaking..
Generated sub-dataframe for 2018JAN from source 1 of 2.
Generated sub-dataframe for 2018FEB from source 1 of 2.
Generated sub-dataframe for 2018MAR from source 1 of 2.
Generated sub-dataframe for 2018APR from source 1 of 2.
Generated sub-dataframe for 2018MAY from source 1 of 2.
Generated sub-dataframe for 2018JUN from source 1 of 2.
Generated sub-dataframe for 2018JUL from source 1 of 2.
Generated sub-dataframe for 2018AUG from source 1 of 2.
Generated sub-dataframe for 2018SEP from source 1 of 2.
Generated sub-dataframe for 2018OCT from source 1 of 2.
Generated sub-dataframe for 2018NOV from source 1 of 2.
Generated sub-dataframe for 2018DEC from source 1 of 2.
Generated sub-dataframe for 2019JAN from source 1 of 2.
Generated sub-dataframe for 2019FEB from source 1 of 2.
Generated sub-dataframe for 2019MAR from source 1 of 2.
Generated sub-dataframe for 2019APR from source 1 of 2.
Generated sub-dataframe for 2019MAY from source 1 of 2.
Generated sub-data